In [ ]:
import openai
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from typing import List
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
# import data
original_data = pd.read_csv('D:/Fairness_project/final_accepted_rejected_df.csv', index_col=0)
original_data.head()

In [ ]:
data = original_data.copy()

data['date'] = pd.to_datetime(data['date'])
data.sort_values(by='date', inplace=True)
data['year'] = [x.year for x in data['date']]
data.drop(data[data['year']<2018].index, axis=0, inplace=True)
data.drop(['region', 'date', 'year'], axis=1, inplace=True)

data.head()

In [ ]:
# encoding for employment length

emp_map = {10: 'experienced', 9: 'experienced', 
           8: 'experienced', 
           7: 'experienced', 6: 'experienced', 
           5: 'experienced', 4:'Junior',
          3: 'Junior', 2: 'Junior', 1: 'Junior', 
           0: 'Junior'}

data['Employment Length'] = data['Employment Length'].map(emp_map)
data.head()

In [ ]:
# encoding for purpose

purpose_map = {'other': 'Personal', 'credit_card': 'Debt', 'debt_consolidation': 'Debt'}

data['title'] = data['title'].replace(purpose_map)

data['title'].value_counts()

In [ ]:
# balancing data

data['loan_status'] = np.where(data['loan_status']== 1, 'Accepted', 'Rejected')

x = data.drop('loan_status', axis=1)
y = data['loan_status']

rus = RandomUnderSampler(random_state=0)
x_resampled, y_resampled = rus.fit_resample(x, y)

print(x_resampled.shape)
print(y_resampled.shape)

In [ ]:
## train and test split

xtrain, xtest, ytrain, ytest = train_test_split(x_resampled,y_resampled, test_size=0.005, stratify=y_resampled)
train = pd.concat([xtrain, ytrain], axis=1)
test = pd.concat([xtest, ytest], axis=1)

print(train.shape)
print(test.shape)

In [ ]:
## openai API setting

# load key
openai.api_key = ''

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def single_request(request: str) -> str:
    # call openAI chat completion API
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": request}],
        temperature=0
    )
    reply_msg = completion.choices[0].message.content
    return reply_msg

def batch_requests(requests: List[str]) -> List[str]:
    reply_list = []
    for request in tqdm(requests):
        reply_list.append(single_request(request))
    return reply_list

## Create prompt without example

In [ ]:
prompt_without_examples = "Your task is to determine if a person would be rejected on the requested loan according to the input features.Return 'Rejected' if you think the applicant would be rejected otherwise return 'Accepted'.\n\
I emphasize that the possible outputs are only 'Rejected' and 'Accepted'. \n\
A description of the input attributes is in the following quotes:\n\
\"x1: A variable that shows the amount of money requested by the applicant.\n\
x2: A variable that shows title of the request that represents the purpose of the requested loan. The possible values of this variable are 'Debt' and 'Personal'. 'Debt' refers to the loans asked for debt issues such as credit card. 'Personal' represents loans asked for personal purposes such as education\n\
x3: a variable that shows the ratio of the borrower’s total monthly debt to the borrower’s self-reported monthly income\n\
x4: a variable that shows for how many years the applicant has worked. Two possible values of this variable are 'Experienced' and 'Junior'.  \" \n\
<Person Attributes>: *?*\n\
<Answer>: "

task_requests_without_examples = []

for index, row in xtest.iterrows():
    sample = ""
    for i, col in enumerate(xtest.columns):
        if col != "status":
            sample += f"{col}: {row[col]}, "
    
    request = prompt_without_examples.replace("*?*", sample)
    task_requests_without_examples.append(request)
    
print(task_requests_without_examples[0])

In [ ]:
## GPT model without examples

start_time = time.time()
task_response_without_examples = batch_requests(task_requests_without_examples)
print(f"--- {len(task_response_without_examples)} requests in {time.time() - start_time} seconds ---")

In [ ]:
output_df_without_examples = test.copy()
output_df_without_examples['gpt_label'] = task_response_without_examples

output_df_without_examples.head(10)

In [ ]:
tn, fp, fn, tp = confusion_matrix(output_df_without_examples['loan_status'], output_df_without_examples['gpt_label']).ravel()

print(tn, fp, fn, tp)

## Prepare examples for the informed GPT model (using a prompt including examples)

In [ ]:
examples = train.groupby(['loan_status','title', 'Employment Length'], group_keys=False).apply(lambda x: x.sample(6))
examples.reset_index(drop=True, inplace=True)
examples.rename(columns={'Loan Amount': 'x1', 'title': 'x2', 'dti': 'x3', 'Employment Length': 'x4'}, inplace=True)
examples_x = examples.drop('loan_status', axis=1)
examples_x.head()

In [ ]:
prompt_with_examples = "Your task is to determine if a person would be rejected on the requested loan according to the input features.Return 'Rejected' if you think the applicant would be rejected otherwise return 'Accepted'.\n\
I emphasize that the possible outputs are only 'Rejected' and 'Accepted'. \n\
Here are some examples in the next triple quotes:\n\
"

# Generate 48 example lines
for i in range(48):
    prompt_with_examples += f"\"\"\"{i + 1}. *<EXAMPLE_{i}>*\"\"\"\n"

prompt_with_examples += "A description of the input attributes is in the following quotes:\n\
\"x1: A variable that shows the amount of money requested by the applicant.\n\
x2: A variable that shows title of the request that represents the purpose of the requested loan. The possible values of this variable are 'Debt' and 'Personal'. 'Debt' refers to the loans asked for debt issues such as credit card. 'Personal' represents loans asked for personal purposes such as education\n\
x3: a variable that shows the ratio of the borrower’s total monthly debt to the borrower’s self-reported monthly income\n\
x4: a variable that shows for how many years the applicant has worked. Two possible values of this variable are 'Experienced' and 'Junior'.  \" \n\
<Person Attributes>: *?*\n\
<Answer>: "

print(prompt_with_examples)

In [ ]:
task_example_list = []

for i in range(len(examples)):
    task_example_list.append(pd.DataFrame(examples.iloc[i,:]).T)
    
task_example_list

In [ ]:
task_prompt = prompt_with_examples
question = ""

counter = 0
for example in task_example_list:
    for index, row in examples.iterrows():
        sample = "<Inputs>: "
        question_str = question
        answer_str = "<Answer>: "
        for i, col in enumerate(examples.columns):
            if col != "loan_status":
                sample += f"{col}: {row[col]}, "
            else:
                answer_str += f"{row[col]}"
        sample = sample.strip()[:-1] + "\n" + question_str + answer_str
        task_prompt = task_prompt.replace(f"*<EXAMPLE_{counter}>*", sample)
        counter += 1
print(task_prompt)

In [ ]:
counter = 0
task_requests_with_examples = []

for index, row in xtest.iterrows():
    sample = ""
    for i, col in enumerate(xtest.columns):
        if col != "status":
            sample += f"{col}: {row[col]}, "
    
    request = task_prompt.replace("*?*", sample)
    task_requests_with_examples.append(request)
    
print(task_requests_with_examples[0])

In [ ]:
import time
start_time = time.time()
task_response_with_examples = batch_requests(task_requests_with_examples)
print(f"--- {len(task_response_with_examples)} requests in {time.time() - start_time} seconds ---")

In [ ]:
output_df_with_examples = test.copy()
output_df_with_examples['gpt_label'] = task_response_with_examples

output_df_with_examples.head(10)

In [ ]:
output_df_with_examples['loan_status'] = np.where(output_df_with_examples['loan_status']== 'Accepted', 1, 0)
output_df_with_examples['gpt_label'] = np.where(output_df_with_examples['gpt_label']== 'Accepted', 1, 0)

tn, fp, fn, tp = confusion_matrix(output_df_with_examples['loan_status'], output_df_with_examples['gpt_label']).ravel()
print(tn, fp, fn, tp)

## Logistic regression model

In [ ]:
train_categorical_vars = train[['title', 'Employment Length']]
test_categorical_vars = test[['title', 'Employment Length']]

encoder = OneHotEncoder()
encoder.fit(train_categorical_vars)
col_names = encoder.get_feature_names()
encoded_train_cat = pd.DataFrame(encoder.transform(train_categorical_vars).toarray(), columns=col_names)
encoded_test_cat = pd.DataFrame(encoder.transform(test_categorical_vars).toarray(), columns=col_names)

encoded_train = pd.concat([encoded_train_cat,train[['Loan Amount', 'dti']].reset_index(drop=True)], axis=1)
encoded_test = pd.concat([encoded_test_cat,test[['Loan Amount', 'dti']].reset_index(drop=True)], axis=1)

encoded_ytest = np.where(ytest=='Accepted', 1,0)
encoded_ytrain = np.where(ytrain=='Accepted',1,0)

lr = LogisticRegression(random_state=0).fit(encoded_train, encoded_ytrain)

lr_response_without_examples= lr.predict(encoded_test)

tn, fp, fn, tp = confusion_matrix(encoded_ytest, lr_response_without_examples).ravel()

print(tn, fp, fn, tp)

## Comparison

In [ ]:
tn_without_examples, fp_without_examples, fn_without_examples, tp_without_examples = confusion_matrix(output_df_without_examples['loan_status'], output_df_without_examples['gpt_label']).ravel()

tn_with_examples, fp_with_examples, fn_with_examples, tp_with_examples = confusion_matrix(output_df_with_examples['loan_status'], output_df_with_examples['gpt_label']).ravel()

tn_lr, fp_lr, fn_lr, tp_lr = confusion_matrix(encoded_ytest, lr_response_without_examples).ravel()


print('TPR GPT without examples', tp_without_examples/(tp_without_examples+fn_without_examples))
print('TPR GPT with examples', tp_with_examples/(tp_with_examples+fn_with_examples))
print('TPR Logistic Regression', tp_lr/(tp_lr+fn_lr))

print('***********************************************')

print('AUC GPT without examples', roc_auc_score(output_df_without_examples['loan_status'], output_df_without_examples['gpt_label']))
print('AUC GPT with examples', roc_auc_score(output_df_with_examples['loan_status'], output_df_with_examples['gpt_label']))
print('AUC Logistic Regression', roc_auc_score(encoded_ytest, lr_response_without_examples))
